
# Task 1 Clean and Upload Data


In [ ]:
import os
import io
from zipfile import ZipFile
import pandas as pd
import csv
from google.cloud import bigquery
from google.oauth2 import service_account

#### Connecting to GBQ

In [ ]:


client = bigquery.Client(project = "wedge_project_np")

### Configure Goggle Big Query

In [ ]:
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job_config.schema_update_options = [bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]

#job_config.skip_leading_rows = 1

#### Opening Zip Files

In [ ]:
for file in os.listdir("data/SmallZip"):
    print(file)

#### Iterate Over Files and Upload

In [ ]:
data_directory = "data/SmallZip/"
zip_files = os.listdir("data/SmallZip")

In [ ]:
def table_exists(client, table_ref):
    try:
        client.get_table(table_ref)
        return True
    except:
        return False

In [ ]:
for file in zip_files :
    with ZipFile(data_directory + file,'r') as zf :  
        print(zf.namelist())

In [ ]:
### How is this different compared to below. I didn't extract below and need to??

#with ZipFile(data_directory+file) as my_zip:
    #print(my_zip.namelist())
    #for zipped_file in my_zip.namelist():
        #x = my_zip.extract(zipped_file)

In [ ]:
#current_zf = zip_files[0]
for current_zf in zip_files :
# Open the current zipfile
    with ZipFile(data_directory + current_zf,'r') as zf :
        zipped_files = zf.namelist()
        
        # Iteraate over each file inside the current zip file
        for file_name in zipped_files :
            # Open and wrap it to read as text
            input_file = io.TextIOWrapper(zf.open(file_name, 'r'), encoding="utf-8")

            for idx, line in enumerate(input_file) :
                print(line)
                if idx > 4 :
                    break
            input_file.close()
        print("\n")


## Sniffing out the Delimiter 

In [ ]:
delimiters = dict() 

# Start by reading in all the files again.

#current_zf = zip_files[0]
for current_zf in zip_files :
# Open the current zf
    with ZipFile(data_directory + current_zf,'r') as zf :
        zipped_files = zf.namelist()
        
        # Iteraate over each file inside the current zip file
        for file_name in zipped_files :
            # Open and wrap it to read as text
            input_file = io.TextIOWrapper(zf.open(file_name, 'r'), encoding="utf-8")
            
            dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
            delimiters[file_name] = dialect.delimiter
            
            print(" ".join(["For",
                           file_name,
                           "the delimiter is",
                           dialect.delimiter
                           ]))

            input_file.close() # tidy up

## Checking for Headers 

In [ ]:
# Create a function to look for headers

def is_header_row(first_row, second_row):
    """
    Function to determine if the first row is a header based on the types of data in the rows.
    Returns True if the first row looks like a header.
    """
    # Check if the first row contains non-numeric values and second row contains numeric values
    for first, second in zip(first_row, second_row):
        # If the first value is not a number but the second value is, it's likely a header
        if first.isdigit() == False and second.isdigit() == True:
            return True
    return False

In [ ]:
headers = dict()
delimiters = dict() 

for current_zf in zip_files :
    with ZipFile(data_directory + current_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            # Open and wrap it to read as text
            input_file = io.TextIOWrapper(zf.open(file_name, 'r'), encoding="utf-8")
            
            dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
            delimiters[file_name] = dialect.delimiter
            
            print(" ".join(["For",
                           file_name,
                           "the delimiter is",
                           dialect.delimiter
                           ]))
            
            this_delimiter = delimiters[file_name]

            # Read the first two lines of the file
            first_line = input_file.readline().strip().split(this_delimiter)
            second_line = input_file.readline().strip().split(this_delimiter)
            
            # Check if the first line is a header row
            has_header = is_header_row(first_line, second_line)
            
            # Print if it has a header
            if has_header:
                print(f"File {file_name} has a header row.")
            else:
                print(f"File {file_name} does NOT have a header row.")
            
            # Append result to headers dictionary
            headers[file_name] = has_header

            input_file.close() # tidy up

## Read through Zip files, find delimeters, find headers

In [ ]:
headers = dict()
delimiters = dict() 

for current_zf in zip_files :
    with ZipFile(data_directory + current_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            # Open and wrap it to read as text
            input_file = io.TextIOWrapper(zf.open(file_name, 'r'), encoding="utf-8")
            
            dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
            delimiters[file_name] = dialect.delimiter
            
            print(" ".join(["For",
                           file_name,
                           "the delimiter is",
                           dialect.delimiter
                           ]))
            
            this_delimiter = delimiters[file_name]

            # Read the first two lines of the file
            first_line = input_file.readline().strip().split(this_delimiter)
            second_line = input_file.readline().strip().split(this_delimiter)
            
            # Check if the first line is a header row
            has_header = is_header_row(first_line, second_line)
            
            # Print if it has a header
            if has_header:
                print(f"File {file_name} has a header row.")
                header_option = 0  # The first line is a header
            else:
                print(f"File {file_name} does NOT have a header row.")
                header_option = None  # No header present

            # Append result to headers dictionary
            headers[file_name] = has_header

            #Read into a dataframe
            try:
                # Read into a DataFrame, assuming the first line might be a header
                df = pd.read_csv(input_file, delimiter=this_delimiter, header = 0)

                # Print the first few rows of the DataFrame to confirm
                print(f"DataFrame from {file_name}:")
                print(df.head())  # Show the first 5 rows of the DataFrame

            except Exception as e:
                # Handle the case where reading fails
                print(f"Error reading {file_name}: {e}")

            input_file.close() # tidy up


In [ ]:
print(df)

### Schema

In [ ]:
job_config.schema = []


In [ ]:
full_path= data_directory + current_zf
if os.path.exists(full_path):
    print("File found")

    with open(data_directory + current_zf, 'rb') as source_file:
        job = client.load_table_from_file(
            source_file,
            f"{gbq_dataset_id}.{current_zf}",
            job_config=job_config,
        )
else:
    print("File not found")

In [ ]:
job.result()

### Create table names for tables in GBQ

In [ ]:

for file in data_directory:
    tab, _ = file.replace("_clean.csv", "")
    table_full_name = ".".join([gbq_project_id, gbq_dataset_id, tab])

    if not table_exists(client, table_full_name):
        table_ref = client.create_table(table=table_full_name)
    else:
        table_ref = client.get_table(table_full_name)

    table = client.get_table(table_ref)
    print("Table {} contains {} columns".format(table_ref.table_id, len(table.schema)))

In [ ]:
tab, _ = file.replace("_clean.csv", "")
table_full_name = ".".join([gbq_project_id, gbq_dataset_id, tab])

if not table_exists(client, table_full_name):
    table_ref = client.create_table(table=table_full_name)
else:
    table_ref = client.get_table(table_full_name)
